# Business Analysis around Product Lifecycles and Seasonality 

### What can I infer about the high-level Seasonality of an eCommerce retailer given a year of transaction-level data?


In [1]:
import pandas as pd
import numpy as np
import re
import datetime
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
##https://www.kaggle.com/vijayuv/onlineretail?select=OnlineRetail.csv

In [2]:
import plotly.express as px
import plotly
import plotly.graph_objs as go
from plotly.offline import iplot
plotly.offline.init_notebook_mode(connected=True)

In [3]:
config = {
  'toImageButtonOptions': {
    'format': 'svg', # one of png, svg, jpeg, webp
    'filename': 'custom_image',
    'height': 800,
    'width': 1150,
    'scale': 1.35 # Multiply title/legend/axis/canvas sizes by this factor
  }
}

In [4]:
pd.set_option('display.max_columns', 50)

In [5]:
#importing the dataset as a df=ttl
ttl = pd.read_csv('/Users/APinkerton/NYC_DS_A/projects/proj_1/OnlineRetail.csv',encoding= 'unicode_escape')
ttl.columns
ttl.head()
#just to get a sense of the countries for the dataset's transactions
# ttl.Country.value_counts()
#Below is jst the counts and totals--91% of the transactions are from the UK
# len(ttl) = 541,909 observations or transactions
# Uk = 495,478 (91%)
# EU ~98%

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [6]:
#getting the unique values of the timestamps of transactions
np.unique(ttl['InvoiceDate'])

array(['1/10/2011 10:04', '1/10/2011 10:07', '1/10/2011 10:08', ...,
       '9/9/2011 9:13', '9/9/2011 9:38', '9/9/2011 9:52'], dtype=object)

In [7]:
#Splitting timestamps from the data in the dataframe--converted and then dropped the orignal 
#expanding to keep all of the info from the timestamp
date = ttl.InvoiceDate.str.split(" ",expand=True)
full = pd.merge(ttl, date, left_index=True, right_index=True)
full.drop(columns=['InvoiceDate',1],inplace=True)
full.head()

,InvoiceNo,StockCode,Description,Quantity,UnitPrice,CustomerID,Country,0
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2.55,17850.0,United Kingdom,12/1/2010
1,536365,71053,WHITE METAL LANTERN,6,3.39,17850.0,United Kingdom,12/1/2010
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2.75,17850.0,United Kingdom,12/1/2010
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,3.39,17850.0,United Kingdom,12/1/2010
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,3.39,17850.0,United Kingdom,12/1/2010


In [8]:
#using the dataset's Unit Price and Quantity to get the sales amount by product
full['Sales_(GBP)']=full['Quantity']*full['UnitPrice']
#Creating a return flag--if a quantity is less than 0, then i classify the boolean of ret (return) as TRUE
#--else FALSE (so a sale)
full['ret'] = full['Quantity'] < 0
full.head()

,InvoiceNo,StockCode,Description,Quantity,UnitPrice,CustomerID,Country,0,Sales_(GBP),ret
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2.55,17850.0,United Kingdom,12/1/2010,15.30,False
1,536365,71053,WHITE METAL LANTERN,6,3.39,17850.0,United Kingdom,12/1/2010,20.34,False
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2.75,17850.0,United Kingdom,12/1/2010,22.00,False
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,3.39,17850.0,United Kingdom,12/1/2010,20.34,False
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,3.39,17850.0,United Kingdom,12/1/2010,20.34,False


In [9]:
#converting to datetime, then converting datetime to new columns with month, day, and year separate
full[0]=pd.to_datetime(full[0])
full['Month']=full[0].dt.month
full['day']=full[0].dt.day
full['year']=full[0].dt.year
full.columns
full.head()

,InvoiceNo,StockCode,Description,Quantity,UnitPrice,CustomerID,Country,0,Sales_(GBP),ret,Month,day,year
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2.55,17850.0,United Kingdom,2010-12-01,15.30,False,12,1,2010
1,536365,71053,WHITE METAL LANTERN,6,3.39,17850.0,United Kingdom,2010-12-01,20.34,False,12,1,2010
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2.75,17850.0,United Kingdom,2010-12-01,22.00,False,12,1,2010
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,3.39,17850.0,United Kingdom,2010-12-01,20.34,False,12,1,2010
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,3.39,17850.0,United Kingdom,2010-12-01,20.34,False,12,1,2010


In [10]:
#I imported a day csv that I created in google sheets to quickly convert
#then dropped those columns into my working dataframe
#Retail companies tend to have a lot of time & date information, so I made sure to have all of that here
##Month is Calendar month--Typical month that anyone is used to seeing
#Day -- same with day
#Year--Same with year
#Wk is using the National Retail Federation's (NRF) 454 Calendar Week--acutally in this case it would be 445, since I am starting the year in January
#454 is a way of structuring the weeks and months so that weeks have the same number of weekdays between years so that they are comparable--always Sun-Sat in a week
#woy is Week of Year--starting with 1 in Jan, 4 weeks in Jan, another 4 in Feb, 5 in Mar, and then quarterly pattern follows the same 445 pattern 
dates = pd.read_csv('/Users/APinkerton/NYC_DS_A/projects/proj_1/days.csv')
dates['date']=pd.to_datetime(dates['date'])
dates.rename(columns={'Unnamed: 2':'woy'},inplace=True)

In [11]:
new = pd.merge(full,dates,left_on=0,right_on='date')
new

,InvoiceNo,StockCode,Description,Quantity,UnitPrice,CustomerID,Country,0,Sales_(GBP),ret,Month,day,year,date,Wk,woy
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2.55,17850.0,United Kingdom,2010-12-01,15.30,False,12,1,2010,2010-12-01,DEC Wk 1,48
1,536365,71053,WHITE METAL LANTERN,6,3.39,17850.0,United Kingdom,2010-12-01,20.34,False,12,1,2010,2010-12-01,DEC Wk 1,48
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2.75,17850.0,United Kingdom,2010-12-01,22.00,False,12,1,2010,2010-12-01,DEC Wk 1,48
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,3.39,17850.0,United Kingdom,2010-12-01,20.34,False,12,1,2010,2010-12-01,DEC Wk 1,48
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,3.39,17850.0,United Kingdom,2010-12-01,20.34,False,12,1,2010,2010-12-01,DEC Wk 1,48
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,0.85,12680.0,France,2011-12-09,10.20,False,12,9,2011,2011-12-09,DEC Wk 2,49
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2.10,12680.0,France,2011-12-09,12.60,False,12,9,2011,2011-12-09,DEC Wk 2,49
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,4.15,12680.0,France,2011-12-09,16.60,False,12,9,2011,2011-12-09,DEC Wk 2,49
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,4.15,12680.0,France,2011-12-09,16.60,False,12,9,2011,2011-12-09,DEC Wk 2,49


In [12]:
#I included 454 Month as a Column, as it sometimes differs from Calendar month
new['454_Month'] = new['Wk'].astype(str).str[0:3]
new.head()

,InvoiceNo,StockCode,Description,Quantity,UnitPrice,CustomerID,Country,0,Sales_(GBP),ret,Month,day,year,date,Wk,woy,454_Month
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2.55,17850.0,United Kingdom,2010-12-01,15.30,False,12,1,2010,2010-12-01,DEC Wk 1,48,DEC
1,536365,71053,WHITE METAL LANTERN,6,3.39,17850.0,United Kingdom,2010-12-01,20.34,False,12,1,2010,2010-12-01,DEC Wk 1,48,DEC
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2.75,17850.0,United Kingdom,2010-12-01,22.00,False,12,1,2010,2010-12-01,DEC Wk 1,48,DEC
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,3.39,17850.0,United Kingdom,2010-12-01,20.34,False,12,1,2010,2010-12-01,DEC Wk 1,48,DEC
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,3.39,17850.0,United Kingdom,2010-12-01,20.34,False,12,1,2010,2010-12-01,DEC Wk 1,48,DEC


In [13]:
#Grouping by Product and then summing sales for the full year
total_top = new.groupby(['StockCode','Description'])[['Sales_(GBP)','Quantity']].agg(['sum'])

In [14]:
#Rearranging and pushing down the column titles to be non-tuples
total_top = total_top.reset_index()
total_top.columns = total_top.columns.get_level_values(0)
total_top.columns

Index(['StockCode', 'Description', 'Sales_(GBP)', 'Quantity'], dtype='object')

In [15]:
#grouping and summing total year sales for all products, sorting best to worst by revenue
total_top.sort_values(by=['Sales_(GBP)'], ascending=False, inplace=True)
all_products = total_top
len(set(all_products['StockCode']))

3958

In [16]:
# Plotting all products in a boxplot. I wanted to try and see what would come from a boxplot for all of the
#products, knowing full well that it would likely end up looking oddThis did not turn out to be an
#insightful visual, so I'm not going to use it. Still including it here for reference
#I am thinking a scatter plot with revenue on one axis and sls units on the other will be more readable
fig7 = go.Figure([
    go.Box(name='All Product Sales',
        x=all_products['Sales_(GBP)'],
        marker=dict(color='red', size=2),
        showlegend=True),
    
])
fig7.update_layout(
    yaxis_title='Sales',
    xaxis_title='Month of Year',
    title='Net Sales by Month',
    hovermode="x")
fig7.update_xaxes(tickmode='linear')
fig7.show(config=config)

In [17]:
for_scatter = all_products[['Description','Sales_(GBP)','Quantity']]
for_scatter.columns

Index(['Description', 'Sales_(GBP)', 'Quantity'], dtype='object')

In [18]:
fig11 = go.Figure(data=go.Scatter(
    x=for_scatter['Sales_(GBP)'],
    y = for_scatter['Quantity'],
    mode='markers',
    marker=dict(
        size=16,
        color='green', 
        colorscale=None,
        showscale=True,
        opacity=.3
        
    )
))

fig11.update_layout(
    yaxis_title='Sales Units',
    xaxis_title='Sales Revenue',
    title='Net Sales Revenue and Units--All Products',
    hovermode="x")

fig11.show(config=config)

In [19]:
##Now I am starting to work with the transation data, and aggregating sales by the dates that I dropped into the DF above
#First grouping by the StockCode/SKU and product description--this is unique by product

#First line is the minimum Week of Year that the product is sold--so a jury rigged 'launch week' essentially
new.groupby(['StockCode','Description'])[['woy']].agg('min')

#Grouping by product by week, making a weekly total DF
new.groupby(['StockCode','Description', 'Wk']).agg('sum')
week_ttl = new.groupby(['Wk'])[['Quantity','Sales_(GBP)']].agg('sum')

#creating a monthly total sales, dropping product info for the monthly totals. I am using monthly totals for reference
cal_month_ttl = new.groupby(['Month'])[['Quantity','Sales_(GBP)']].agg('sum').copy()
cal_month_ttl['Unit_Price']=cal_month_ttl['Sales_(GBP)']/cal_month_ttl['Quantity']

#just getting a value for the total sales & units for month
ttlsls = round(cal_month_ttl['Sales_(GBP)'].sum(),0)
ttlquan = cal_month_ttl['Quantity'].sum()
#tuple of the total sales and quantity
monthly_ttls = (ttlsls,ttlquan)

#putting in columns for each month's percent to total of the year--do this for revenue and units
cal_month_ttl['% to Sls'] = cal_month_ttl['Sales_(GBP)'] / ttlsls
cal_month_ttl['% to Quan'] = cal_month_ttl['Quantity'] / ttlquan

#build is just a month's total revenue or unit number over the previous month's value--so basically a multiplier over the previous month 
#(build is usually at the monthly or weekly level)
cal_month_ttl['build']=round(cal_month_ttl['Sales_(GBP)']/cal_month_ttl['Sales_(GBP)'].shift(1,axis=0),2)
#this was a bit wonky, but I wanted the Jan build to be off of Dec, not null--in a normal company setting, I would calculate Jan's build off the previous Dec
#but I only have one year of data--this is what the warning is coming from
cal_month_ttl['build'][1]= round((cal_month_ttl['Sales_(GBP)'][1])/(cal_month_ttl['Sales_(GBP)'][12]),2)
#aggregating for 454 month
month_454_ttl = new.groupby(['454_Month'])[['Quantity','Sales_(GBP)']].agg('sum')

#just getting a separate DF for monthly total net sales
calendar_month_df = cal_month_ttl.copy()
calendar_month = calendar_month_df.reset_index()
calendar_month
##calendar_month_df = cal month total net sales numbers

<ipython-input-19-67c593728c8a>:30: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Month,Quantity,Sales_(GBP),Unit_Price,% to Sls,% to Quan,build
0,1,308966,560000.260,1.812498,0.057449,0.059687,0.47
1,2,277989,498062.650,1.791663,0.051095,0.053703,0.89
2,3,351872,683267.080,1.941806,0.070095,0.067976,1.37
3,4,289098,493207.121,1.706021,0.050597,0.055849,0.72
4,5,380391,723333.510,1.901553,0.074205,0.073485,1.47
5,6,341623,691123.120,2.023058,0.070901,0.065996,0.96
6,7,391116,681300.111,1.741939,0.069893,0.075557,0.99
7,8,406199,682680.510,1.680655,0.070035,0.078471,1.00
8,9,549817,1019687.622,1.854595,0.104608,0.106215,1.49
9,10,570532,1070704.670,1.876678,0.109841,0.110217,1.05


In [20]:
calendar_month['Sls_Cum_sum'] = calendar_month['% to Sls'].cumsum()
calendar_month

,Month,Quantity,Sales_(GBP),Unit_Price,% to Sls,% to Quan,build,Sls_Cum_sum
0,1,308966,560000.260,1.812498,0.057449,0.059687,0.47,0.057449
1,2,277989,498062.650,1.791663,0.051095,0.053703,0.89,0.108544
2,3,351872,683267.080,1.941806,0.070095,0.067976,1.37,0.178639
3,4,289098,493207.121,1.706021,0.050597,0.055849,0.72,0.229236
4,5,380391,723333.510,1.901553,0.074205,0.073485,1.47,0.303441
5,6,341623,691123.120,2.023058,0.070901,0.065996,0.96,0.374342
6,7,391116,681300.111,1.741939,0.069893,0.075557,0.99,0.444235
7,8,406199,682680.510,1.680655,0.070035,0.078471,1.00,0.514270
8,9,549817,1019687.622,1.854595,0.104608,0.106215,1.49,0.618878
9,10,570532,1070704.670,1.876678,0.109841,0.110217,1.05,0.728719


In [21]:
#Total net revenue and sales units for the business, by calendar month

fig = go.Figure([
    go.Scatter(name='Units Sold',
        x=calendar_month['Month'],
        y=calendar_month['Quantity'],
        mode='markers+lines',
        marker=dict(color='red', size=2),
        showlegend=True),
    
    go.Scatter(name='Sales Rev. (GBP)',
        x=calendar_month['Month'],
        y=calendar_month['Sales_(GBP)'],
        mode='markers+lines',
        marker=dict(color='blue', size=2),
        showlegend=True)
])
fig.update_layout(
    yaxis_title='Sales',
    xaxis_title='Month of Year',
    title='Net Sales by Month',
    hovermode="x")
fig.update_xaxes(tickmode='linear')
fig.show(config=config)

In [22]:
# new.groupby(['Income Group', 'Region'])[['GDP']].agg(np.mean).loc[['High income: OECD', 'High income: nonOECD']]

In [23]:
#this cell is for gross sales calculations--so taking out all returns from sales data. 
#I'm doing pretty much the same calculations fro gross as I did for net

##Gross
maskg = ~new['ret']
gross = new.loc[maskg, :].copy()
gross['Unit_Price']=gross['Sales_(GBP)']/gross['Quantity']

ttlgrossdol = gross['Sales_(GBP)'].sum()
ttlgrossquan = gross['Quantity'].sum()

gross['% to Gross (GBP)'] = gross['Sales_(GBP)'] / ttlgrossdol
gross['% to Gross U'] = gross['Quantity'] / ttlgrossquan

ttl_mo_gross = gross.groupby(['Month'])[['Quantity','Sales_(GBP)','% to Gross (GBP)','% to Gross U']].agg('sum')

ttl_mo_gross['% to Gross U'] = ttl_mo_gross['Quantity'] / ttlgrossdol
monthly_gross_ttls = (ttlgrossdol,ttlgrossquan)
ttl_mo_gross = ttl_mo_gross.reset_index()
##ttl_mo_gross cal month ttl gross sales numbers

In [24]:
#now I am starting to do calculations for returns now (so everything that has a Return = TRUE flag)
#Largely the same as the gross table, but with the Return Flag to true, not false
#return mask
mask1 = new['ret']
returns = new.loc[mask1, :].copy()

#column calculations
returns['Unit_Price']=returns['Sales_(GBP)']/returns['Quantity']

#total return calculations by month
ttlretdol = returns['Sales_(GBP)'].sum()
ttlretquan = returns['Quantity'].sum()

#just getting a percent to total for returns just like with month--
#this will come in handy to demonstrate the dropoff in sales from 
#Dec>Jan can be partially attributed to Holiday/Gifting returns
returns['% to Ret$'] = returns['Sales_(GBP)'] / ttlretdol
returns['% to RetU'] = returns['Quantity'] / ttlretquan

#total monthly returns DF
ttl_mo_ret = returns.groupby(['Month'])[['Quantity','Sales_(GBP)','% to Ret$','% to RetU']].agg('sum')

#More return calculations
ttl_mo_ret['% to RetU'] = ttl_mo_ret['Quantity'] / ttlretdol
monthly_ttls = (ttlretdol,ttlretquan)
# ttl_mo_ret = ttl_mo_ret.reset_index()
##ttl_mo_ret = ttl returns only


In [25]:
#Pulling sales and returns out separately to see how much net seasonality is strictly from sales, and what is
#coming from returns actualizing in possibly a separate month than the original sale
ttl_mo_gross = ttl_mo_gross.rename(columns={'Quantity':'Gross_Quantity', 'Sales_(GBP)' :'Gross_Sales_(GBP)'})
ttl_mo_gross
# ttl_mo_gross = ttl_mo_gross.reset_index()

,Month,Gross_Quantity,Gross_Sales_(GBP),% to Gross (GBP),% to Gross U
0,1,397716,691364.560,0.064950,0.037363
1,2,286695,523631.890,0.049192,0.026933
2,3,384950,717639.360,0.067418,0.036164
3,4,312176,537808.621,0.050524,0.029327
4,5,399425,770536.020,0.072388,0.037524
5,6,394337,761739.900,0.071561,0.037046
6,7,407539,719221.191,0.067567,0.038286
7,8,425016,737014.260,0.069239,0.039928
8,9,575416,1058590.172,0.099449,0.054057
9,10,628745,1154979.300,0.108504,0.059067


In [26]:
#Now pulling together returns to compare to gross sales
ttl_mo_ret= ttl_mo_ret.rename(columns={'Quantity':'Ret_Quantity', 'Sales_(GBP)' :'Ret_Rev_(GBP)'})

In [75]:
#Putting gross sales and returns in the same DF--easier to pull into the same graph this way
gro_ret = pd.concat([ttl_mo_gross,ttl_mo_ret],axis=1)
# gro_ret = gro_ret.reset_index()
# gro_ret['return_%_U']=gro_ret['']
gro_ret['Return_U_%'] = gro_ret['Ret_Quantity'] / gro_ret['Gross_Quantity']
gro_ret['Return_Rev_%'] = gro_ret['Ret_Rev_(GBP)'] / gro_ret['Gross_Sales_(GBP)']
# gro_ret.columns =['Month', 'Gross_Quantity', 'Gross_Sales_(GBP)', '% to Gross (GBP)',
#         '% to Gross U', 'Ret_Quantity', 'Ret_Rev_(GBP)', '% to Ret$',
#         '% to RetU', 'Return_U_%', 'Return_Rev_%']
# ttl_mo_ret = ttl_mo_ret.reset_index()
ttl_mo_ret

,Month,Ret_Quantity,Ret_Rev_(GBP),% to Ret$,% to RetU
0,1,-88750,-131364.30,0.146479,0.098962
1,2,-8706,-25569.24,0.028511,0.009708
2,3,-33078,-34372.28,0.038327,0.036884
3,4,-23078,-44601.50,0.049733,0.025733
4,5,-19034,-47202.51,0.052634,0.021224
5,6,-52714,-70616.78,0.078742,0.058779
6,7,-16423,-37921.08,0.042284,0.018313
7,8,-18817,-54333.75,0.060585,0.020982
8,9,-25599,-38902.55,0.043379,0.028544
9,10,-58213,-84274.63,0.093971,0.064911


In [76]:
gro_ret

,Month,Gross_Quantity,Gross_Sales_(GBP),% to Gross (GBP),% to Gross U,Month,Ret_Quantity,Ret_Rev_(GBP),% to Ret$,% to RetU,Return_U_%,Return_Rev_%
0,1,397716,691364.560,0.064950,0.037363,1,-88750,-131364.30,0.146479,0.098962,-0.223149,-0.190007
1,2,286695,523631.890,0.049192,0.026933,2,-8706,-25569.24,0.028511,0.009708,-0.030367,-0.048831
2,3,384950,717639.360,0.067418,0.036164,3,-33078,-34372.28,0.038327,0.036884,-0.085928,-0.047896
3,4,312176,537808.621,0.050524,0.029327,4,-23078,-44601.50,0.049733,0.025733,-0.073926,-0.082932
4,5,399425,770536.020,0.072388,0.037524,5,-19034,-47202.51,0.052634,0.021224,-0.047654,-0.061259
5,6,394337,761739.900,0.071561,0.037046,6,-52714,-70616.78,0.078742,0.058779,-0.133678,-0.092705
6,7,407539,719221.191,0.067567,0.038286,7,-16423,-37921.08,0.042284,0.018313,-0.040298,-0.052725
7,8,425016,737014.260,0.069239,0.039928,8,-18817,-54333.75,0.060585,0.020982,-0.044274,-0.073721
8,9,575416,1058590.172,0.099449,0.054057,9,-25599,-38902.55,0.043379,0.028544,-0.044488,-0.036749
9,10,628745,1154979.300,0.108504,0.059067,10,-58213,-84274.63,0.093971,0.064911,-0.092586,-0.072966


In [77]:
#Plotting gross sales/units and returns on the same graph to geta more complete look at seasonal and monthly
#changes and builds in sales and returns

fig2 = go.Figure([
    go.Scatter(name='Gross Units Sold',
        x=ttl_mo_gross['Month'],
        y=ttl_mo_gross['Gross_Quantity'],
        mode='markers+lines',
        marker=dict(color='orange', size=2),
        showlegend=True),
    
    go.Scatter(name='Gross Sales Rev. (GBP)',
        x=ttl_mo_gross['Month'],
        y=ttl_mo_gross['Gross_Sales_(GBP)'],
        mode='markers+lines',
        marker=dict(color='green', size=2),
        showlegend=True),
    
    go.Scatter(name='Return Units',
        x=ttl_mo_ret['Month'],
        y=ttl_mo_ret['Ret_Quantity'],
        mode='markers+lines',
        marker=dict(color='purple', size=2),
        showlegend=True),

    go.Scatter(name='Return Rev.(GBP)',
        x=ttl_mo_ret['Month'],
        y=ttl_mo_ret['Ret_Rev_(GBP)'],
        mode='markers+lines',
        marker=dict(color='magenta', size=2),
        showlegend=True)
])
fig2.update_layout(
    yaxis_title='Sales/Return Values',
    xaxis_title='Month of Year',
    title='Gross Sales/Return Transactions by Month',
    hovermode="x")
fig2.update_xaxes(tickmode='linear')
fig2.show(config=config)

In [89]:
#Creating a return rate calculation
gro_ret['Return_U_%'] = abs(gro_ret['Return_U_%'])
gro_ret['Return_Rev_%'] = abs(gro_ret['Return_Rev_%'])
gro_ret
mo_list=[1,2,3,4,5,6,7,8,9,10,11,12]

In [90]:
#charting that same return rate % by month by itself
fig3 = go.Figure([
    go.Scatter(name='Return Rate (Units)',
        x=mo_list,
        y=gro_ret['Return_U_%'],
        mode='markers+lines',
        marker=dict(color='orange', size=2),
        showlegend=True),
    
    go.Scatter(name='Return Rate (Rev. in GBP)',
        x=mo_list,
        y=gro_ret['Return_Rev_%'],
        mode='markers+lines',
        marker=dict(color='green', size=2),
        showlegend=True),
    
])
fig3.update_layout(
    yaxis_title='Return Rates by Month (% to Gross)',
    xaxis_title='Month of Year',
    title='Return Rate by Month',
    hovermode="x")
fig3.update_xaxes(tickmode='linear')
fig3.show()

In [32]:
new.columns

Index([  'InvoiceNo',   'StockCode', 'Description',    'Quantity',
         'UnitPrice',  'CustomerID',     'Country',             0,
       'Sales_(GBP)',         'ret',       'Month',         'day',
              'year',        'date',          'Wk',         'woy',
         '454_Month'],
      dtype='object')

In [33]:
#Now I am starting to get into product level analysis, first splitting into products > 10k GBP in rev 
#(there are 217)--just getting a sense of the product level 

prod_tbl = new.groupby(['StockCode','Description'])[['Sales_(GBP)','Quantity']].agg('sum')
above10 = prod_tbl.sort_values(by='Sales_(GBP)',ascending=False).head(217)
topsls = above10['Sales_(GBP)'].sum()
topquan = above10['Quantity'].sum()
prod_tbl

Sales_(GBP)  Quantity
StockCode    Description                                               
10002        INFLATABLE POLITICAL GLOBE                759.89       860
10080        GROOVY CACTUS INFLATABLE                  119.09       303
             check                                       0.00        22
10120        DOGGY RUBBER                               40.53       193
10123C       HEARTS WRAPPING TAPE                        3.25         5
...                                                       ...       ...
gift_0001_20 to push order througha s stock was          0.00        10
gift_0001_30 Dotcomgiftshop Gift Voucher £30.00        175.53         7
gift_0001_40 Dotcomgiftshop Gift Voucher £40.00        100.70         3
gift_0001_50 Dotcomgiftshop Gift Voucher £50.00        167.56         4
m            Manual                                      2.55         1

[4792 rows x 2 columns]

In [34]:
#this is the rest of the products (below 10k GBP in revenue in the yr)
below10 = prod_tbl.sort_values(by='Sales_(GBP)',ascending=False).head(~217)
botsls = below10['Sales_(GBP)'].sum()
botquan = below10['Quantity'].sum()
print(topsls/botsls,topquan/botquan)

0.4722343549120401 0.33655731768860775


In [35]:
#i'm creating a list of Stock Codes and the Unit price associated with them to try and back into a markdown flag
mds = new[['StockCode','UnitPrice']]
mds = mds.apply(tuple,axis=1)
md = pd.DataFrame(set(mds))
md.sort_values(by=[0,1],axis=1,inplace=True, ascending=False)
# md.to_csv('md.csv')
#I exported the Stock Code and unit price, then sorted by StockCode and then Price highest to lowest 
#each product in the csv, then reimported--i used the highest price as the regular price
#for the sake of comapring performance when products are marked down or on promo, I ignored styles that \
#weren't marked down during this period

price_tbl = pd.read_csv('/Users/APinkerton/NYC_DS_A/projects/proj_1/mds.csv')
price_tbl.head()

,StockCode,Unit_Price,Price_Flag,Num_of_Pr,Ever_Marked
0,10002,1.66,Reg_Price,4,Marked
1,10002,1.63,MD,4,Marked
2,10002,0.85,MD,4,Marked
3,10002,0.00,MD,4,Marked
4,10080,0.85,Reg_Price,3,Marked


In [36]:
#making copies of the total sale table, bringing in the above flags that i created in one table
price_perf = new.copy()
price_perf.head()
pricing_sales = price_perf.merge(price_tbl, how='left', left_on=['StockCode','UnitPrice'], right_on=['StockCode','Unit_Price'])
pricing_sales.head()
# mask1 = pricing_sales['Ever_Marked']=='Marked'
# mask2 = pricing_sales['ret']==False
# pricing_sales.loc[mask2,:]
reg_md = pricing_sales.copy()
reg_md.head()

,InvoiceNo,StockCode,Description,Quantity,UnitPrice,CustomerID,Country,0,Sales_(GBP),ret,Month,day,year,date,Wk,woy,454_Month,Unit_Price,Price_Flag,Num_of_Pr,Ever_Marked
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2.55,17850.0,United Kingdom,2010-12-01,15.30,False,12,1,2010,2010-12-01,DEC Wk 1,48,DEC,2.55,MD,10,Marked
1,536365,71053,WHITE METAL LANTERN,6,3.39,17850.0,United Kingdom,2010-12-01,20.34,False,12,1,2010,2010-12-01,DEC Wk 1,48,DEC,3.39,MD,4,Marked
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2.75,17850.0,United Kingdom,2010-12-01,22.00,False,12,1,2010,2010-12-01,DEC Wk 1,48,DEC,2.75,MD,11,Marked
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,3.39,17850.0,United Kingdom,2010-12-01,20.34,False,12,1,2010,2010-12-01,DEC Wk 1,48,DEC,3.39,MD,8,Marked
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,3.39,17850.0,United Kingdom,2010-12-01,20.34,False,12,1,2010,2010-12-01,DEC Wk 1,48,DEC,3.39,MD,12,Marked


In [37]:
#I wanted to get a weekly Reg Price v MD total and Avg table just to get a more detailed look at how
#Reg price things sell versus Reg, what really drives the business
final_pricing = reg_md.groupby(['Month','Price_Flag'])[['Sales_(GBP)','Quantity']].agg(['sum','count']).copy()
final_pricing['Avg_Qty'] = final_pricing[('Quantity', 'sum')] / final_pricing[('Quantity', 'count')]
final_pricing['Avg_Sls_(GBP)'] = final_pricing[('Sales_(GBP)', 'sum')] / final_pricing[('Sales_(GBP)', 'count')]
final_pricing = final_pricing.reset_index()
final_pricing.columns = final_pricing.columns.get_level_values(0)
final_pricing

,Month,Price_Flag,Sales_(GBP),Sales_(GBP),Quantity,Quantity,Avg_Qty,Avg_Sls_(GBP)
0,1,MD,536620.750,32711,294652,32711,9.007734,16.404902
1,1,Reg_Price,23379.510,2436,14314,2436,5.876026,9.597500
2,2,MD,483494.890,26326,263541,26326,10.010674,18.365680
3,2,Reg_Price,14567.760,1381,14448,1381,10.461984,10.548704
4,3,MD,647752.950,34777,341053,34777,9.806855,18.625901
5,3,Reg_Price,35514.130,1971,10819,1971,5.489092,18.018331
6,4,MD,480540.750,28404,276857,28404,9.747113,16.918066
7,4,Reg_Price,12666.371,1512,12241,1512,8.095899,8.377229
8,5,MD,704032.780,35300,367299,35300,10.405071,19.944271
9,5,Reg_Price,19300.730,1730,13092,1730,7.567630,11.156491


In [38]:
final_pricing.columns
final_pricing.head()

,Month,Price_Flag,Sales_(GBP),Sales_(GBP),Quantity,Quantity,Avg_Qty,Avg_Sls_(GBP)
0,1,MD,536620.75,32711,294652,32711,9.007734,16.404902
1,1,Reg_Price,23379.51,2436,14314,2436,5.876026,9.597500
2,2,MD,483494.89,26326,263541,26326,10.010674,18.365680
3,2,Reg_Price,14567.76,1381,14448,1381,10.461984,10.548704
4,3,MD,647752.95,34777,341053,34777,9.806855,18.625901


In [39]:
#Splitting the aggregated pricing table into Reg Price and Markdown
final_pricing_reg_m = final_pricing['Price_Flag']=='Reg_Price'
final_pricing_reg = final_pricing.loc[final_pricing_reg_m]
final_pricing_md_m = final_pricing['Price_Flag']=='MD'
final_pricing_md = final_pricing.loc[final_pricing_md_m]
final_pricing_md
final_pricing_reg.columns=['Month','Price_Flag','Sales_(GBP)','x','Quantity','y','Avg_Qty', 'Avg_Sls_(GBP)']
final_pricing_reg
final_pricing_md.columns=['Month','Price_Flag','Sales_(GBP)','x','Quantity','y','Avg_Qty', 'Avg_Sls_(GBP)']
final_pricing_md['Sales_(GBP)'].sum()

9236491.421000045

In [40]:
print(511256.51/9236491.42)

0.05535180911801248


In [41]:
#Graphing Reg price and Markdown Sales revenue and unit sales
fig4 = go.Figure([
    go.Scatter(name='Reg Price Total Rev.(GBP)',
        x=final_pricing_reg['Month'],
        y=final_pricing_reg['Sales_(GBP)'],
        mode='markers+lines',
        marker=dict(color='pink', size=2),
        showlegend=True),
    
    go.Scatter(name='Reg Price Sales Units',
        x=final_pricing_reg['Month'],
        y=final_pricing_reg['Quantity'],
        mode='markers+lines',
        marker=dict(color='violet', size=2),
        showlegend=True),

    go.Scatter(name='Markdown Total Rev.(GBP)',
        x=final_pricing_md['Month'],
        y=final_pricing_md['Sales_(GBP)'],
        mode='markers+lines',
        marker=dict(color='blue', size=2),
        showlegend=True),
    
    go.Scatter(name='Markdown Sales Units',
        x=final_pricing_md['Month'],
        y=final_pricing_md['Quantity'],
        mode='markers+lines',
        marker=dict(color='maroon', size=2),
        showlegend=True),
    
])
fig4.update_layout(
    yaxis_title='Sales',
    xaxis_title='Month of Year',
    title='Reg Price/Markdown Sales by Month',
    hovermode="x")
fig4.update_xaxes(tickmode='linear')
fig4.show(config=config)

In [42]:
#Now I want to break the pricing table into only products that were marked down at some point during the year
#and look at how they performed on average before vs after the pricing change
md_mask = reg_md['Ever_Marked']=='Marked'
before_after_md = reg_md.loc[md_mask, :]
before_after_md
b_a = before_after_md.groupby(['StockCode', 'woy', 'Price_Flag'])[['Sales_(GBP)', 'Quantity']].agg(['sum','count']).copy()
b_a['Week_Count'] = 1
b_a = b_a.reset_index()
b_a.groupby(['Price_Flag'])[[('Sales_(GBP)', 'sum'), ('Quantity','sum'), ('Week_Count', "")]].agg(['sum'])


,Sales_(GBP),Quantity,Week_Count
,sum,sum,
,sum,sum,sum
Price_Flag,,,
MD,9.236491e+06,4926637,91678
Reg_Price,4.898729e+05,240201,18000


In [43]:
b_a['Avg_Wkly_Sls'] = b_a[('Sales_(GBP)','sum')] / b_a[('Week_Count', '')] 
b_a['Avg_Wkly_Quan'] = b_a[('Quantity','sum')] / b_a[('Week_Count', '')] 
b_a
pricing_product_perf = b_a.groupby(['Price_Flag'])[[('Avg_Wkly_Sls',''), ('Avg_Wkly_Quan',"")]].agg(['mean']).copy()
pricing_product_perf['Unit_Price'] = pricing_product_perf[('Avg_Wkly_Sls','')] / pricing_product_perf[('Avg_Wkly_Quan',"")]

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:2877: PerformanceWarning:

indexing past lexsort depth may impact performance.



In [44]:
import warnings
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

In [45]:
new

,InvoiceNo,StockCode,Description,Quantity,UnitPrice,CustomerID,Country,0,Sales_(GBP),ret,Month,day,year,date,Wk,woy,454_Month
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2.55,17850.0,United Kingdom,2010-12-01,15.30,False,12,1,2010,2010-12-01,DEC Wk 1,48,DEC
1,536365,71053,WHITE METAL LANTERN,6,3.39,17850.0,United Kingdom,2010-12-01,20.34,False,12,1,2010,2010-12-01,DEC Wk 1,48,DEC
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2.75,17850.0,United Kingdom,2010-12-01,22.00,False,12,1,2010,2010-12-01,DEC Wk 1,48,DEC
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,3.39,17850.0,United Kingdom,2010-12-01,20.34,False,12,1,2010,2010-12-01,DEC Wk 1,48,DEC
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,3.39,17850.0,United Kingdom,2010-12-01,20.34,False,12,1,2010,2010-12-01,DEC Wk 1,48,DEC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,0.85,12680.0,France,2011-12-09,10.20,False,12,9,2011,2011-12-09,DEC Wk 2,49,DEC
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2.10,12680.0,France,2011-12-09,12.60,False,12,9,2011,2011-12-09,DEC Wk 2,49,DEC
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,4.15,12680.0,France,2011-12-09,16.60,False,12,9,2011,2011-12-09,DEC Wk 2,49,DEC
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,4.15,12680.0,France,2011-12-09,16.60,False,12,9,2011,2011-12-09,DEC Wk 2,49,DEC


In [46]:
by_prod = new.groupby(['StockCode','Description','Month'])[['Quantity','Sales_(GBP)']].agg(['sum']).copy()

In [47]:
#Getting the list of the top 25 products here, and splitting out the rest of the products into a separate tbl
top_week = new.groupby(['StockCode','Description','woy'])[['Quantity','Sales_(GBP)']].agg(['sum']).copy()
top = new.groupby(['StockCode','Description'])[['Quantity','Sales_(GBP)']].agg(['sum']).copy()
rest=[]
top_prod = top.sort_values(('Sales_(GBP)', 'sum'),ascending=False).head(25).reset_index().copy()
rest_prod = top.sort_values(('Sales_(GBP)', 'sum'),ascending=False).iloc[25:,:].reset_index().copy()
top25=[]
for d in top_prod.Description:
    top25.append(d)
for d in rest_prod.Description:
    rest.append(d)

top25
rest
top_prod.columns = top_prod.columns.get_level_values(0)
top_week.reset_index(inplace=True)
top_week.columns = top_week.columns.get_level_values(0)
boolean_ser = top_week.Description.isin(top25)
top_week = top_week.loc[boolean_ser,:]

In [48]:
top_week.head()

,StockCode,Description,woy,Quantity,Sales_(GBP)
4348,20725,LUNCH BAG RED RETROSPOT,1,338,523.02
4349,20725,LUNCH BAG RED RETROSPOT,2,353,557.41
4350,20725,LUNCH BAG RED RETROSPOT,3,240,383.44
4351,20725,LUNCH BAG RED RETROSPOT,4,165,324.97
4352,20725,LUNCH BAG RED RETROSPOT,5,417,735.76


In [50]:
#Getting the calendar month totals and top 25 products in the same place to compare them to one another
#Not nearly as skewed as I assumed the sales would turn out--top 25 products are 16% of total yearly sales (gbp)
#and only 8% of the sales units (top 25 are 60 basis points of the total SKU's sold in the year to be fair)
print(25/3958)
calendar_month[['Sales_(GBP)','Quantity']].sum()
#9,747,747.93;5,176,450
# top25
prod_tbl = prod_tbl.reset_index()
mas = prod_tbl['Description'].isin(top25)
top_25 = prod_tbl.loc[mas,:]
top_25[['Sales_(GBP)','Quantity']].sum()
#1511419.8;399660.0
print(1511419/9747747.93)
print(399660/5176450)

0.006316321374431531
0.15505314774794346
0.07720735252924302


In [67]:
fig13 = px.line(top_week, x="woy", y="Sales_(GBP)", color="Description",
              line_group="Description", hover_name="Description")
fig13.update_layout(
    yaxis_title='Sales Rev.(GBP)',
    xaxis_title='Week of Year',
    title='Top 25 (by Rev.) Product Performance',
    hovermode="x")
fig13.update_xaxes(tickmode='linear')
fig13.show()

In [71]:
fig13.to_json()

'{"data":[{"hoverlabel":{"namelength":0},"hovertemplate":"<b>%{hovertext}</b><br><br>Description=LUNCH BAG RED RETROSPOT<br>woy=%{x}<br>Sales_(GBP)=%{y}","hovertext":["LUNCH BAG RED RETROSPOT","LUNCH BAG RED RETROSPOT","LUNCH BAG RED RETROSPOT","LUNCH BAG RED RETROSPOT","LUNCH BAG RED RETROSPOT","LUNCH BAG RED RETROSPOT","LUNCH BAG RED RETROSPOT","LUNCH BAG RED RETROSPOT","LUNCH BAG RED RETROSPOT","LUNCH BAG RED RETROSPOT","LUNCH BAG RED RETROSPOT","LUNCH BAG RED RETROSPOT","LUNCH BAG RED RETROSPOT","LUNCH BAG RED RETROSPOT","LUNCH BAG RED RETROSPOT","LUNCH BAG RED RETROSPOT","LUNCH BAG RED RETROSPOT","LUNCH BAG RED RETROSPOT","LUNCH BAG RED RETROSPOT","LUNCH BAG RED RETROSPOT","LUNCH BAG RED RETROSPOT","LUNCH BAG RED RETROSPOT","LUNCH BAG RED RETROSPOT","LUNCH BAG RED RETROSPOT","LUNCH BAG RED RETROSPOT","LUNCH BAG RED RETROSPOT","LUNCH BAG RED RETROSPOT","LUNCH BAG RED RETROSPOT","LUNCH BAG RED RETROSPOT","LUNCH BAG RED RETROSPOT","LUNCH BAG RED RETROSPOT","LUNCH BAG RED RETROSPOT","

In [53]:
#I want to chart sales across a product's life regardless of what week of the year the product launched
#so here I am starting to group by product and the weeks that there were transactions against the product
first_wk = new.copy()
first_week = first_wk.groupby(['StockCode','Description','woy'])[['Quantity', 'Sales_(GBP)']].agg(['sum']).copy()
# first_week.columns = first_week.columns.get_level_values(0)

In [54]:
#Now I wanted to get the minimum for the week of the year that there were transactions against a certain product
#again, this is not the most ideal way to get a launch period, but definitely the best that I can do only
#having transaction data

first_week =  first_week.reset_index()
first_week.columns = first_week.columns.get_level_values(0)
min_week = first_week.groupby(['StockCode', 'Description'])[[('woy')]].agg(['min']).copy()
min_week

woy
                                                 min
StockCode    Description                            
10002        INFLATABLE POLITICAL GLOBE            1
10080        GROOVY CACTUS INFLATABLE              9
             check                                45
10120        DOGGY RUBBER                          8
10123C       HEARTS WRAPPING TAPE                  9
...                                               ..
gift_0001_20 to push order througha s stock was   24
gift_0001_30 Dotcomgiftshop Gift Voucher £30.00    1
gift_0001_40 Dotcomgiftshop Gift Voucher £40.00   14
gift_0001_50 Dotcomgiftshop Gift Voucher £50.00   18
m            Manual                               51

[4792 rows x 1 columns]

In [55]:
#Dropping the "launch week/minimum transaction week" into the full product table--getting what week of the 
#product's life that specific week is by subtracting the two (and adding one)
min_week.columns = min_week.columns.get_level_values(0)
min_week = min_week.rename(columns={'woy':'f_wol'})
min_week

f_wol
StockCode    Description                               
10002        INFLATABLE POLITICAL GLOBE               1
10080        GROOVY CACTUS INFLATABLE                 9
             check                                   45
10120        DOGGY RUBBER                             8
10123C       HEARTS WRAPPING TAPE                     9
...                                                 ...
gift_0001_20 to push order througha s stock was      24
gift_0001_30 Dotcomgiftshop Gift Voucher £30.00       1
gift_0001_40 Dotcomgiftshop Gift Voucher £40.00      14
gift_0001_50 Dotcomgiftshop Gift Voucher £50.00      18
m            Manual                                  51

[4792 rows x 1 columns]

In [56]:
#I am excluding products that showed up for the first time in the first 8 weeks of the year--because
#the data starts in Late Dec/early Jan it's likely that these products weren't acutally launched the first week
#they show up in transactions, I don't want to include them. Not many new products are launched in Jan anyway,
#regardless of retailer
#I wanted to also exclude products that were launched in the last 9 weeks of the year also, since I knew that it 
#wouldn't give the best or fullest representation of a typical product lifecycle on the year, and would spike
#the first 9 weeks in a misleading way
m1 = min_week['f_wol']>8
m2 = min_week['f_wol']<44
first_week = min_week.loc[m1 & m2,:].copy()
first_week

,,f_wol
StockCode,Description,
10080,GROOVY CACTUS INFLATABLE,9
10123C,HEARTS WRAPPING TAPE,9
10124G,ARMY CAMO BOOKCOVER TAPE,11
15030,FAN BLACK FRAME,12
15058A,BLUE POLKADOT GARDEN PARASOL,9
...,...,...
PADS,PADS TO MATCH ALL CUSHIONS,15
gift_0001_10,Dotcomgiftshop Gift Voucher £10.00,10
gift_0001_20,to push order througha s stock was,24


In [57]:
#Putting together a table for avg. Sales revenue and units by product week of life
plm_track = new.merge(first_week, how='left', left_on='Description', right_on='Description')
prod_to_wol = plm_track['ret']==False
intermed = plm_track.loc[prod_to_wol,:]

plm = intermed.groupby(['Description','woy','f_wol'])[['Sales_(GBP)','Quantity']].agg(['sum']).copy()
plm = plm.reset_index()
plm.columns=plm.columns.get_level_values(0)

plm['prod_wol'] = plm['woy'] - plm['f_wol'] + 1
lifecycle = plm.groupby(['prod_wol'])[['Sales_(GBP)','Quantity']].agg(['mean'])

prod_mask = plm['prod_wol']>0
lifecycle = plm.loc[prod_mask,:]
life_tbl = lifecycle.groupby(['prod_wol'])[['Sales_(GBP)','Quantity']].agg(['mean'])
life_tbl = life_tbl.reset_index()
life_tbl.columns = life_tbl.columns.get_level_values(0)
life_tbl.head()

,prod_wol,Sales_(GBP),Quantity
0,1.0,129.309048,76.531328
1,2.0,165.060529,84.176471
2,3.0,145.530257,86.089124
3,4.0,143.866780,78.010786
4,5.0,149.729832,87.961832


In [68]:
#Charting the product week of life avg sales revenue and units

fig5 = go.Figure([
    go.Scatter(name='Avg. Wkly Sls Rev.',
        x=life_tbl['prod_wol'],
        y=life_tbl['Sales_(GBP)'],
        mode='markers+lines',
        marker=dict(color='red', size=2),
        showlegend=True),
    
    go.Scatter(name='Avg. Wkly SlsU',
        x=life_tbl['prod_wol'],
        y=life_tbl['Quantity'],
        mode='markers+lines',
        marker=dict(color='blue', size=2),
        showlegend=True)

])
fig5.update_layout(
    yaxis_title='Avg. Sales per Week',
    xaxis_title='Week of Product\'s Life (1=Launch Week)',
    title='Sales curve post-Launch',
    hovermode="x")
fig5.update_xaxes(tickmode='linear')
fig5.show()

In [72]:
# fig5.to_json()